# *-- Data Mining -- Milestone 2*
##### Management of data (Group)

## 1. Stock Data 
### Crawl companies name

https://www.thestar.com.my/business/marketwatch/

In [1]:
# import package
from selenium import webdriver

browser = webdriver.Chrome() # open web page
browser.implicitly_wait(10) # wait for web page to load

company_names = [] # save company names in a list
# crawl all company names from A-Z
for i in range(65,91):
    url = 'https://www.thestar.com.my/business/marketwatch/stock-list/?alphabet='+chr(i)
    browser.get(url)
    
    name_list = browser.find_elements_by_xpath('//table[@class="market-trans"]//tr[@class="linedlist"]/td/a')
    for name in name_list:
        if name.text!='':
            name_text = name.text.replace("&","%26") 
            company_names.append(name_text)

# crawl all company names in 0-9
url1 = 'https://www.thestar.com.my/business/marketwatch/stock-list/?alphabet=0-9'
browser.get(url1)
name1_list = browser.find_elements_by_xpath('//table[@class="market-trans"]//tr[@class="linedlist"]/td/a')
for name1 in name1_list:
    if name1.text!='':
        name1_text = name1.text.replace("&","%26") 
        company_names.append(name1_text)

print(company_names)
browser.close()

# save as links for crawling all the information
company_links = []
for n in company_names:
    link = 'https://www.thestar.com.my/business/marketwatch/stocks/?qcounter=' + n
    company_links.append(link)

#print(company_links)

['A50CHIN-C26', 'A50CHIN-C28', 'A50CHIN-C30', 'A50CHIN-C32', 'A50CHIN-C34', 'A50CHIN-C36', 'A50CHIN-C38', 'A50CHIN-C40', 'A50CHIN-H25', 'A50CHIN-H27', 'A50CHIN-H29', 'AASIA', 'AAX', 'AAX-WA', 'ABFMY1', 'ABLEGRP', 'ABMB', 'ABMB-CY', 'ABMB-CZ', 'ACME', 'ACOSTEC', 'ADVCON', 'ADVENTA', 'ADVPKG', 'AEM', 'AEM-WA', 'AEMULUS', 'AEON', 'AEON-CO', 'AEON-CQ', 'AEON-CR', 'AEONCR', 'AEONCR-CC', 'AEONCR-LA', 'AFFIN', 'AFUJIYA', 'AHB', 'AHB-WB', 'AHEALTH', 'AHP', 'AIM', 'AIRASIA', 'AIRASIAC71', 'AIRASIAC72', 'AIRASIAC74', 'AIRASIAC75', 'AIRASIAC76', 'AIRASIAC77', 'AIRASIAC78', 'AIRASIAC79', 'AIRASIAC80', 'AIRASIAC81', 'AIRASIAC82', 'AIRASIAC83', 'AIRASIAC84', 'AIRPORT', 'AIRPORT-C4', 'AIRPORT-C5', 'AIRPORT-C6', 'AIRPORT-C7', 'AIRPORT-C8', 'AISB', 'AJI', 'AJIYA', 'AJIYA-WA', 'ALAM', 'ALAM-WA', 'ALAQAR', 'ALCOM', 'ALLIANZ', 'ALLIANZ-PA', 'ALSREIT', 'AMBANK', 'AMBANK-C10', 'AMBANK-C11', 'AMBANK-C13', 'AMBANK-C14', 'AMBANK-C15', 'AMBANK-C16', 'AMEDIA', 'AMFIRST', 'AMLEX', 'AMPROP', 'AMPROP-PA', 'AMPROP-P

### Crawl companies sector information

In [5]:
# import package
from selenium import webdriver
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

name_list = []
code_list = []
sector_1_list = []
sector_2_list = []

sector_list = []
sector_name_list = []
sector_elements = []
sector_links = []
company_names = []
company_links = []

header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
        'Referer':'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-2305666475781689&output=html&h=90&slotname=9390552809&adk=3116781417&adf=1662620477&w=970&lmt=1552753083&guci=2.2.0.0.2.2.0.0&format=970x90&url=http%3A%2F%2Fwww.investalks.com%2Fforum%2Fforum.php%3Fmod%3Dforumdisplay%26fid%3D7%26filter%3Dtypeid%26typeid%3D17&flash=0&wgl=1&dt=1552753083912&bpp=42&bdt=108&fdt=46&idt=22&shv=r20190313&cbv=r20190131&saldr=aa&abxe=1&correlator=6683659723222&frm=20&pv=2&ga_vid=691979923.1552569252&ga_sid=1552752744&ga_hid=1232295072&ga_fc=1&iag=0&icsg=12206&dssz=10&mdo=0&mso=0&u_tz=480&u_his=3&u_java=0&u_h=864&u_w=1536&u_ah=824&u_aw=1536&u_cd=24&u_nplug=3&u_nmime=4&adx=34&ady=100&biw=1026&bih=350&scr_x=0&scr_y=0&eid=21060853&oid=3&rx=0&eae=0&fc=656&brdim=426%2C33%2C426%2C33%2C1536%2C0%2C1057%2C735%2C1042%2C350&vis=1&rsz=%7C%7CeE%7C&abl=CS&ppjl=f&pfx=0&fu=16&bc=7&ifi=1&uci=1.afatrway4czq&fsb=1&xpc=kkAgITwzNl&p=http%3A//www.investalks.com&dtd=90'}

def get_sectors():
    # set up browser
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(options=chrome_options)
    # browser = webdriver.Chrome() # open web page
    browser.implicitly_wait(10)  # wait for web page to load

    url = 'https://www.thestar.com.my/business/marketwatch/'
    browser.get(url)
    r = browser.page_source
    html = BeautifulSoup(r, 'html.parser')
    # print(html)
    browser.close()

    # sector elements
    htmlPart = html.find(class_=re.compile("stocks"))
    linkPart = [x.get_attribute_list('id') for x in htmlPart.find_all('a', {"id": True})]
    for i in range(len(linkPart)):
        sector_elements.extend(linkPart[i])
    # print(linkPart)
    # print(sector_elements)
    # print(len(sector_elements))

    # sector_list
    sector = html.find_all('strong')
    for i in sector:
        sector_list.append(i.text.strip(':'))
        # print(i.text)
    # print(sector_list)

    # sector_name_list
    sector_n = html.select('div.text a')
    for i in sector_n:
        sector_name_list.append(i.text)
    # print(sector_name_list)

    return


def get_company_names():
    for l in sector_list:
        for e, n in zip(sector_elements, sector_name_list):
            if l.lower()[0] == e[0]:
                # set up browser
                chrome_options = webdriver.ChromeOptions()
                prefs = {"profile.managed_default_content_settings.images": 2}
                chrome_options.add_experimental_option("prefs", prefs)
                browser = webdriver.Chrome(options=chrome_options)
                # browser = webdriver.Chrome() # open web page
                browser.implicitly_wait(10)  # wait for web page to load

                url_s = 'https://www.thestar.com.my/business/marketwatch/stock-list/?sector=' + e
                # sector_links.append(url_s)
                browser.get(url_s)
                name_list = browser.find_elements_by_xpath(
                    '//table[@class="market-trans"]//tr[@class="linedlist"]/td/a')
                for name in name_list:
                    if name.text != '':
                        name_text = name.text.replace("&", "%26")
                        company_names.append(name_text)
                        sector_1_list.append(l)
                        sector_2_list.append(n)
                browser.close()

    # save as links for crawling all the information
    for n in company_names:
        link = 'https://www.thestar.com.my/business/marketwatch/stocks/?qcounter=' + n
        company_links.append(link)

    return

class AppCrawler:
    def __init__(self, starting_url, depth):
        self.starting_url = starting_url
        self.depth = depth
        self.apps = []

    def crawl(self):
        self.get_app_from_link(self.starting_url)
        return

    def get_app_from_link(self, link):
        start_page = requests.get(link,headers = header)
        tree = html.fromstring(start_page.text)

        name = tree.xpath('//h1[@class="stock-profile f16"]/text()')[0]
        code = tree.xpath('//li[@class="f14"]/text()')[1]

        name_list.append(name)
        print(name)
        code_list.append(code[3:])
        print(code[3:])

        return


class App:

    def __init__(self, name, code, links):
        self.name = name
        self.code = code
        self.links = links

    def __str__(self):
        return ("Name: " + self.name.encode('UTF-8') +
                "\r\nCode: " + self.developer.encode('UTF-8') + "\r\n")


get_sectors()
get_company_names()

for link_c in company_links:
    crawler = AppCrawler(link_c, 0)
    crawler.crawl()


ADVENTA BHD
7191
APEX HEALTHCARE BHD
7090
DUOPHARMA BIOTECH BERHAD
7148
HARTALEGA HOLDINGS BHD
5168
IHH HEALTHCARE BERHAD
5225
KOSSAN RUBBER INDUSTRIES BHD
7153
KOTRA INDUSTRIES BHD
0002
KPJ HEALTHCARE BHD
5878
PHARMANIAGA BHD
7081
SUPERMAX CORPORATION BHD
7106
TMC LIFE SCIENCES BHD
0101
TMC LIFE SCIENCES BERHAD WARRANTS B
0101WB
TOP GLOVE CORPORATION BHD
7113
Y.S.P.SOUTHEAST ASIA HOLDING
7178
ALAM MARITIM RESOURCES BHD
5115
ALAM MARITIM RESOURCES BERHAD - WARRANTS 2019/2022
5115WA
BUMI ARMADA BERHAD
5210
BARAKAH OFFSHORE PETROLEUM BHD
7251
CARIMIN PETROLEUM BERHAD
5257
COASTAL CONTRACTS BHD
5071
DAYA MATERIALS BHD
0091
DAYANG ENTERPRISE HOLDINGS BHD
5141
DELEUM BHD
5132
DIALOG GROUP BHD
7277
HANDAL RESOURCES BERHAD
7253
HENGYUAN REFINING COMPANY BERHAD
4324
HIBISCUS PETROLEUM BHD
5199
HIBISCUS PETROLEUM BERHAD - WARRANTS 2018/2021
5199WC
SINO HUA-AN INTERNATIONAL BHD
2739
ICON OFFSHORE BERHAD
5255
KNM GROUP BHD
7164
KNM GROUP BERHAD - WARRANTS 2015/2020
7164WB
MALAYSIA MARINE AND HEAV

GAS MALAYSIA BERHAD
5209
MALAKOFF CORPORATION BERHAD
5264
MEGA FIRST CORPORATION BHD
3069
MEGA FIRST CORPORATION BERHAD - WARRANTS A 2016/2020
3069WA
PBA HOLDINGS BHD
5041
PETRONAS GAS BHD
6033
RANHILL HOLDINGS BERHAD
5272
SALCON BHD
8567
SALCON BERHAD - WARRANTS B 2018/2025
8567WB
TALIWORKS CORPORATION BHD
8524
TENAGA NASIONAL BHD
5347
YTL CORPORATION BHD
4677
YTL POWER INTERNATIONAL BHD
6742
ALLIANCE BANK MALAYSIA BERHAD
2488
AEON CREDIT SERVICE (M) BHD
5139
AFFIN BANK BERHAD
5185
ALLIANZ MALAYSIA BHD
1163
ALLIANZ MALAYSIA BHD-ICPS
1163PA
AMMB HOLDINGS BHD
1015
APEX EQUITY HOLDINGS BHD
5088
BIMB HOLDINGS BHD
5258
BIMB HOLDINGS BERHAD-WA 13/23
5258WA
BURSA MALAYSIA BHD
1818
CIMB GROUP HOLDINGS BERHAD
1023
ECM LIBRA FINANCIAL GRP BHD
2143
ELK-DESA RESOURCES BERHAD
5228
HONG LEONG BANK BHD
5819
HONG LEONG CAPITAL BERHAD
5274
HONG LEONG FINANCIAL GROUP BHD
1082
INSAS BHD
3379
INSAS BERHAD-PREFERENCE SHARES A
3379PA
INSAS BERHAD - WB 15/20
3379WB
JOHAN HOLDINGS BHD
3441
KENANGA INVESTMENT

TEK SENG HOLDINGS BERHAD - WARRANTS 2015/2020
7200WA
TEO SENG CAPITAL BHD
7252
TEO SENG CAPITAL BERHAD - Warrants 2015/2020
7252WA
TEO GUAN LEE CORPORATION BHD
9369
TOMEI CONSOLIDATED BHD
7230
TPC PLUS BHD
7176
TPC PLUS BERHAD - WARRANTS A 2016/2021
7176WA
UMW HOLDINGS BHD
4588
UPA CORPORATION BHD
7757
VERTICE BERHAD
7240
VERTICE BERHAD - WARRANTS A 2014/2024
7240WA
WANG-ZHENG BHD
7203
WARISAN TC HOLDINGS BHD
5016
XIDELANG HOLDINGS LTD
5156
XIDELANG HOLDINGS LTD - WARRANTS 2019/2022
5156WD
XIAN LENG HOLDINGS BHD
7121
XINGQUAN INT SPORTS HLDG LTD
5155
XINGQUAN INT SPORTS HLDG LTD-WARRANT 14/19
5155WA
XINGQUAN INTERNATIONAL SPORTS HOLDINGS LIMITED-WARRANTS 2016/2021
5155WB
YEE LEE CORPORATION BHD
5584
YOONG ONN CORPORATION BERHAD
5159
ZHULIAN CORPORATION BHD
5131
ADVANCECON HOLDINGS BERHAD
5281
AHMAD ZAKI RESOURCES BHD
7078
AHMAD ZAKI RESOURCES BERHAD- WA 14/24
7078WA
BENALEC HOLDINGS BERHAD
5190
BINA PURI HOLDINGS BHD
5932
BREM HOLDINGS BHD
8761
CREST BUILDER HOLDINGS BHD
8591
DKLS INDU

HUME INDUSTRIES BHD
5000
HO WAH GENTING BHD
9601
HO WAH GENTING BERHAD-WARRANTS 2016/2021
9601WD
IMASPRO CORPORATION BHD
7222
IPMUDA BHD
5673
IRE-TEX CORPORATION BHD
7183
IRE-TEX CORPORATION BERHAD-WARRANTS 14/19
7183WA
JASA KITA BHD
8648
JCBNEXT BERHAD
0058
KUMPULAN JETSON BHD
9083
JMR CONGLOMERATION BHD
7043
KARYON INDUSTRIES BHD
0054
KEIN HING INTERNATIONAL BHD
7199
KUMPULAN FIMA BHD
6491
KELINGTON GROUP BERHAD
0151
KELINGTON GROUP BERHAD - WARRANTS A
0151WA
KIA LIM BHD
6211
KIM HIN INDUSTRY BHD
5371
KINSTEEL BHD
5060
KKB ENGINEERING BHD
9466
KNUSFORD BHD
5035
KOBAY TECHNOLOGY BHD
6971
KOMARKCORP BHD
7017
KOMARKCORP BERHAD-WARRANTS B 2015/2020
7017WB
KUMPULAN PERANGSANG SELANGOR
5843
KPS CONSORTIUM BHD
9121
KECK SENG (M) BHD
3476
K. SENG SENG CORPORATION BHD
5192
KUB MALAYSIA BHD
6874
KYM HOLDINGS BHD
8362
LAFARGE MALAYSIA BERHAD
3794
LB ALUMINIUM BHD
9326
LOTTE CHEMICAL TITAN HOLDING BERHAD
5284
LEON FUAT BERHAD
5232
LEWEKO RESOURCES BHD
8745
LEWEKO RESOURCES BERHAD - WARRANTS B 20

EDUSPEC HOLDINGS BERHAD
0107
GENETEC TECHNOLOGY BERHAD
0104
G NEPTUNE BERHAD
0045
iDIMENSION CONSOLIDATED BHD
0174
iDIMENSION CONSOLIDATED BERHAD-WARRANTS A
0174WA
IFCA MSC BHD
0023
INIX TECHNOLOGIES HOLDINGS BHD
0094
INIX TECHNOLOGIES HOLDINGS BERHAD - WARRANTS A 2015/2020
0094WA
IRIS CORPORATION BHD
0010
JF TECHNOLOGY BHD
0146
JHM CONSOLIDATION BHD
0127
K-ONE TECHNOLOGY BHD
0111
K-ONE TECHNOLOGY BERHAD - WARRANTS 2018/2021
0111WC
KEY ALLIANCE GROUP BERHAD
0036
KEY ALLIANCE GROUP BERHAD-WARRANTS B 2017/2020
0036WB
KRONOLOGI ASIA BERHAD
0176
LAMBO GROUP BERHAD
0018
LAMBO GROUP BERHAD - WARRANTS 2019/2024
0018WB
LYC HEALTHCARE BERHAD
0075
MICROLINK SOLUTIONS BHD
0126
MIKRO MSC BERHAD
0112
MLABS SYSTEMS BHD
0085
MLABS SYSTEMS-WARRANTS 10/20
0085WA
MLABS SYSTEMS BERHAD - WARRANTS 2017/2020
0085WB
MMAG HOLDINGS BERHAD
0034
MMAG HOLDINGS BERHAD - ICPS A 2017/2022
0034PA
MMAG HOLDINGS BERHAD - WARRANTS B 2017/2022
0034WB
MANAGEPAY SYSTEMS BERHAD
0156
MQ TECHNOLOGY BHD
0070
MQ TECHNOLOGY BERH

SUPERMX-C62: CW SUPERMAX CORPORATION BERHAD (MACQ)
710662
SUPERMX-C63: CW SUPERMAX CORPORATION BERHAD (KIBB)
710663
SUPERMX-C64: CW SUPERMAX CORPORATION BERHAD (AM)
710664
SUPERMX-C65: CW SUPERMAX CORPORATION BERHAD (RHB)
710665
SUPERMX-C66: CW SUPERMAX CORPORATION BERHAD (CIMB)
710666
SUPERMX-C67: CW SUPERMAX CORPORATION BERHAD (MACQ)
710667
SUPERMX-C68: CW SUPERMAX CORPORATION BERHAD (MIBB)
710668
SUPERMX-C69: CW SUPERMAX CORPORATION BERHAD (CIMB)
710669
SUPERMX-C70: CW SUPERMAX CORPORATION BERHAD (RHB)
710670
TOPGLOV-C41: CW TOP GLOVE CORPORATION BHD (CIMB)
711341
TOPGLOV-C42: CW TOP GLOVE CORPORATION BERHAD (MACQ)
711342
TOPGLOV-C43: CW TOP GLOVE CORPORATION BERHAD (KIBB)
711343
TOPGLOV-C44: CW TOP GLOVE CORPORATION BERHAD (MACQ)
711344
TOPGLOV-C45: CW TOP GLOVE CORPORATION BERHAD (MIBB)
711345
TOPGLOV-C46: CW TOP GLOVE CORPORATION BERHAD (MACQ)
711346
TOPGLOV-C47: CW TOP GLOVE CORPORATION BERHAD (RHB)
711347
TOPGLOV-C48: CW TOP GLOVE CORPORATION BERHAD (CIMB)
711348
TOPGLOV-C49: C

MBSB-C16: CW MALAYSIA BUILDING SOCIETY BERHAD (MIBB)
117116
MBSB-C17: CW MALAYSIA BUILDING SOCIETY BERHAD (CIMB)
117117
MBSB-C18: CW MALAYSIA BUILDING SOCIETY BERHAD (MACQ)
117118
MBSB-C20: CW MALAYSIA BUILDING SOCIETY BERHAD (MACQ)
117120
MBSB-C21: CW MALAYSIA BUILDING SOCIETY BERHAD (CIMB)
117121
MBSB-C22: CW MALAYSIA BUILDING SOCIETY BERHAD (KIBB)
117122
MBSB-C23: CW MALAYSIA BUILDING SOCIETY BERHAD (MACQ)
117123
MBSB-C24: CW MALAYSIA BUILDING SOCIETY BERHAD (CIMB)
117124
PBBANK-C27: CW PUBLIC BANK BERHAD (CIMB)
129527
PBBANK-C28: CW PUBLIC BANK BERHAD (MIBB)
129528
PBBANK-C29: CW PUBLIC BANK BERHAD (KIBB)
129529
PBBANK-C30: CW PUBLIC BANK BERHAD (RHB)
129530
PBBANK-C31: CW PUBLIC BANK BERHAD (MACQ)
129531
PBBANK-C32: CW PUBLIC BANK BERHAD (MACQ)
129532
PBBANK-C33: CW PUBLIC BANK BERHAD (CIMB)
129533
PBBANK-C34: CW PUBLIC BANK BERHAD (RHB)
129534
PBBANK-C35: CW PUBLIC BANK BERHAD (KIBB)
129535
RHBBANKC10: CW RHB BANK BERHAD (MIBB)
106610
RHBBANKC12: CW RHB BANK BERHAD (MACQ)
106612


INARI-C30: CW INARI AMERTRON BERHAD (AM)
016630
INARI-C31: CW INARI AMERTRON BERHAD (KIBB)
016631
INARI-C32: CW INARI AMERTRON BERHAD (MACQ)
016632
INARI-C33: CW INARI AMERTRON BERHAD (CIMB)
016633
INARI-C34: CW INARI AMERTRON BERHAD (MIBB)
016634
INARI-C35: CW INARI AMERTRON BERHAD (KIBB)
016635
INARI-C36: CW INARI AMERTRON BERHAD (AM)
016636
INARI-C37: CW INARI AMERTRON BERHAD (CIMB)
016637
INARI-C38: CW INARI AMERTRON BERHAD (RHB)
016638
INARI-C39: CW INARI AMERTRON BERHAD (MIBB)
016639
INARI-C40: CW INARI AMERTRON BERHAD (MACQ)
016640
INARI-C41: CW INARI AMERTRON BERHAD (RHB)
016641
MI-CA: CW MI TECHNOVATION BERHAD (RHB)
5286CA
MI-CB: CW MI TECHNOVATION BERHAD (MACQ)
5286CB
MI-CC: CW MI TECHNOVATION BERHAD (MACQ)
5286CC
MI-CD: CW MI TECHNOVATION BERHAD (RHB)
5286CD
MI-CE: CW MI TECHNOVATION BERHAD (MIBB)
5286CE
MI-CF: CW MI TECHNOVATION BERHAD (KIBB)
5286CF
MYEG-C50: CW MY E.G. SERVICES BERHAD (MIBB)
013850
MYEG-C53: CW MY E.G. SERVICES BERHAD (CIMB)
013853
MYEG-C55: CW MY E.G. SER

In [7]:
na = name_list
co = code_list
s1 = sector_1_list
s2 = sector_2_list

dataframe = pd.DataFrame({'name': na, 'code': co, 'Sector_main': s1, 'sector': s2})

# save data
# dataframe.to_csv("sector.csv", index=False, sep=',')
print(dataframe)


                                                   name    code  Sector_main  \
0                                           ADVENTA BHD    7191  Main Market   
1                                   APEX HEALTHCARE BHD    7090  Main Market   
2                              DUOPHARMA BIOTECH BERHAD    7148  Main Market   
3                                HARTALEGA HOLDINGS BHD    5168  Main Market   
4                                 IHH HEALTHCARE BERHAD    5225  Main Market   
5                          KOSSAN RUBBER INDUSTRIES BHD    7153  Main Market   
6                                  KOTRA INDUSTRIES BHD    0002  Main Market   
7                                    KPJ HEALTHCARE BHD    5878  Main Market   
8                                       PHARMANIAGA BHD    7081  Main Market   
9                              SUPERMAX CORPORATION BHD    7106  Main Market   
10                                TMC LIFE SCIENCES BHD    0101  Main Market   
11                  TMC LIFE SCIENCES BE

### Crawl stock financial reports

  

https://www.klsescreener.com/v2/financial-reports

In [16]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

page = requests.get('https://www.klsescreener.com/v2/financial-reports')
soup = BeautifulSoup(page.text, 'html.parser')
stock_table = soup.find(class_='table table-striped') #specify the location
urls = []
for a in stock_table.find_all('a'):   #find all 'a' from the specified location
    urls.append(a.attrs['href'])      #extract attribute inside 'a'

even_list = [] 
for i in range (0,len(urls)-1):       #only links at even number index is required
    if i % 2 == 0:
        even_list.append(urls[i])

link_front = 'https://www.klsescreener.com'      
even_link = [link_front + s for s in even_list]       #add https:// to the links

fullname_list = []
name_list = []
category_list = []
w52_list = []
ROE_list = []
PE_list = []
EPS_list = []
DPS_list = []
DY_list = []
PTBV_list = []
RPS_list = []
PSR_list = []
Market_Cap_list = []
RSI_list = []
Stochastic_list = []


driver = webdriver.Chrome()

for l in even_link:
        driver.get(l)
        driver.implicitly_wait(10)
        fullname = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[1]/span')
        name = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[1]')
        category = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/span[2]')
        w52 = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[7]/td[2]')
        ROE = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[8]/td[2]')
        PE = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[9]/td[2]')
        EPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[10]/td[2]')
        DPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[11]/td[2]')
        DY = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[12]/td[2]')
        PTBV = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[14]/td[2]')
        RPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[15]/td[2]')
        PSR = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[16]/td[2]')
        Market_Cap = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[17]/td[2]')
        RSI = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[3]/tbody/tr[1]/td[2]')
        Stochastic = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[3]/tbody/tr[2]/td[2]')
        
        fullname1 = fullname.text
        fullname_list.append(fullname1)
        
        name1 = name.text
        name_list.append(name1)
        
        category1 = category.text
        category_list.append(category1)
        
        
        w521 = w52.text
        w52_list.append(w521)
        
        ROE1 = ROE.text
        ROE_list.append(ROE1)
        
        PE1 = PE.text
        PE_list.append(PE1)
        
        EPS1 = EPS.text
        EPS_list.append(EPS1)
        
        DPS1 = DPS.text
        DPS_list.append(DPS1)
        
        DY1 = DY.text
        DY_list.append(DY1)
        
        PTBV1 = PTBV.text
        PTBV_list.append(PTBV1)
        
        RPS1 = RPS.text
        RPS_list.append(RPS1)
        
        PSR1 = PSR.text
        PSR_list.append(PSR1)
        
        Market_Cap1 = Market_Cap.text
        Market_Cap_list.append(Market_Cap1)
        
        RSI1 = RSI.text
        RSI_list.append(RSI1)
        
        Stochastic1 = Stochastic.text
        Stochastic_list.append(Stochastic1)
        
        

import pandas as pd

dataframe = pd.DataFrame({'full name':fullname_list,'name':name_list,'category':category_list,'52w':w52_list, 
                          'ROE':ROE_list, 'P/E':PE_list,'EPS':EPS_list, 'DPS':DPS_list, 'DY':DY_list, 
                          'PTBV':PTBV_list, 'RPS':RPS_list, 'PSR':PSR_list, 'Market_Cap':Market_Cap_list, 
                          'RSI':RSI_list, 'Stochastic':Stochastic_list})

# dataframe.to_csv('stock_klse.csv', index = False, sep = ',') 
print(dataframe)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="page"]/div[2]/div[1]/div[1]/span"}
  (Session info: chrome=74.0.3729.169)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)


In [17]:
driver = webdriver.Chrome()
driver.get(even_link[0])
driver.implicitly_wait(10)
fullname = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[1]/span')
name = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[1]')
category = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/span[2]')
w52 = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[7]/td[2]')
ROE = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[8]/td[2]')
PE = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[9]/td[2]')
EPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[10]/td[2]')
DPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[11]/td[2]')
DY = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[12]/td[2]')
PTBV = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[14]/td[2]')
RPS = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[15]/td[2]')
PSR = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[16]/td[2]')
Market_Cap = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[1]/tbody/tr[17]/td[2]')
RSI = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[3]/tbody/tr[1]/td[2]')
Stochastic = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[2]/div[1]/div/table[3]/tbody/tr[2]/td[2]')

fullname1 = fullname.text
print(fullname1)
name1 = name.text
print(name1)
category1 = category.text
print(category1)
w521 = w52.text
print(w521)
ROE1 = ROE.text
print(ROE1)
PE1 = PE.text
print(PE1)
EPS1 = EPS.text
print(EPS1)
DPS1 = DPS.text
print(DPS1)
DY1 = DY.text
print(DY1)
PTBV1 = PTBV.text
print(PTBV1)
RPS1 = RPS.text
print(RPS1)
PSR1 = PSR.text
print(PSR1)
Market_Cap1 = Market_Cap.text
print(Market_Cap1)
RSI1 = RSI.text
print(RSI1)
Stochastic1 = Stochastic.text
print(Stochastic1)

PASDEC HOLDINGS BERHAD
PASDEC (6912) [s]
PASDEC HOLDINGS BERHAD Summary
Property - Main Market
0.300 - 0.565
-5.39
-8
-4.58
0.00
0.00%
0.41
42.86
1
138.1M
Neutral 40.0
Oversold 20.0


## 2. Currency Data
### Crewl currency data

http://www.bnm.gov.my/?tpl=exchangerates

In [18]:
from selenium import webdriver
import pandas as pd

driver = webdriver.Chrome()
driver.get("http://www.bnm.gov.my/?tpl=exchangerates")

currency = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[2]")
date = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[1]/th[2]/b")
buy = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[3]")
sell = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[5]")
middle = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[7]")


currency_unit = currency.text
date_day = date.text
buy_rate = buy.text
sell_rate = sell.text
middle_rate = middle.text

dataframe = pd.DataFrame({"currency_unit":currency_unit, "date_day":date_day, "buy_rate":buy_rate, "sell_rate":sell_rate, "middle_rate":middle_rate}, index = [0])
# dataframe.to_csv("currency.csv",index=False,sep=',')
print(dataframe)


  currency_unit              date_day buy_rate sell_rate middle_rate
0           USD  31 May 2019 (Friday)   4.1890    4.1920      4.1905


## 3. News Data
### Crawl stock news

https://www.klsescreener.com/v2/news

In [14]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

browser = webdriver.Chrome()  # open web page
browser.implicitly_wait(10)
url='https://www.klsescreener.com/v2/'
browser.get(url)
screen_button=browser.find_element_by_id('submit')
screen_button.click()
#wait until the browser load after clicking the screen button and obtain the new web source
wait=WebDriverWait(browser,10)
element=wait.until(EC.presence_of_element_located((By.CLASS_NAME,"table-responsive")))
html=browser.page_source
soup = BeautifulSoup(html, "html.parser")
#print(soup.prettify())

#locate the result table
stock_table=soup.find(id="result")

#create a list for codes
links = []
for a in stock_table:
    print(a)
    rowsodd = stock_table.find_all(attrs={'class':'list odd'})
    rowseven = stock_table.find_all(attrs={'class':'list even'})
    for item in rowsodd:
        stock_code =item.find('td',attrs={'title':'Code'}).text
        links.append(stock_code)
    for item in rowseven:
        stock_code =item.find('td',attrs={'title':'Code'}).text
        links.append(stock_code)

#remove duplicates
mylinks = list(dict.fromkeys(links))

#add the code to form news urls
company_links = []
for n in mylinks:
    link = 'https://www.klsescreener.com/v2/news/stock/' + n
    company_links.append(link)

#extract news and respective date
df=pd.DataFrame([])
for l in company_links:
    company_news = []
    company_date=[]
    code = l
    page = requests.get(l)
    soup = BeautifulSoup(page.text, 'html.parser')
    news_table=soup.find(id="section")
    for b in news_table.find_all('h2',{"class":"figcaption"}):
        print(b.get_text())
        company_news.append(b.get_text())
        
    for c in news_table.find_all('div',{"class":"item-title-secondary subtitle"}):
        company_date.append(c.get_text())
    dataframe=pd.DataFrame({'code':code, 'date':company_date,'news':company_news})
    df=df.append(dataframe)
    
#save to csv
# df.to_csv("news.csv",index=False,sep=',')
print(df)




<b>954</b>
 stock(s) found.


<div class="table-responsive">
<div aria-hidden="true" class="floatThead-container" style="overflow: hidden; position: fixed; margin-top: 0px; top: 0px; z-index: 1001; will-change: transform; transform: translateX(15px) translateY(485px); left: 0px; width: 1208px;"><table class="tablesorter-bootstrap stocks tablesorter table table-bordered table-striped floatThead-table" style="border-collapse: collapse; border-width: 1px 1px 0px; border-style: solid; border-color: rgb(221, 221, 221); border-image: initial; display: table; width: 1208px; margin: 0px; table-layout: fixed;"><colgroup><col style="width: 111px;"/><col style="width: 71px;"/><col style="width: 143px;"/><col style="width: 70px;"/><col style="width: 75px;"/><col style="width: 81px;"/><col style="width: 67px;"/><col style="width: 62px;"/><col style="width: 59px;"/><col style="width: 79px;"/><col style="width: 55px;"/><col style="width: 67px;"/><col style="width: 68px;"/><col style="width: 77px;"/

Short Position
热门股:数码网络上挑RM5.18
马股五连升 收于1650点上方
KLCI closes at more than two month high, Tenaga powers
KLCI climbs for five straight days, ends week at above 1,650 level
KLCI bucks regional trend, rises 0.37% as Tenaga & PetDag lift
KLCI closes at three week high as Public Bank climbs
No major hiccups in Axiata-Telenor merger — Jamaludin
亚通承诺合并后不裁员
亚通承诺合并后不裁员
Bursa bucks key Asian markets as Tenaga, IHH lift KLCI
与挪威Telenor纯粹合并 亚通否认出售大马业务
Blue chips close at day's high on Tenaga, Maxis, Public Bank's gains
My Say: A robust partner for a connected Asia
Axiata dismisses claims of selling Malaysian interests to Norwegians in planned union with Telenor
数码网络与中兴通讯联手发展5G技术
Sime Plantation, Public Bank nudge KLCI above 1,600
Digi, ZTE to explore potential of 5G technology
Digi Telecommunications in MoU with ZTE to explore 5G technology in Malaysia
KLCI firmer at midday, ringgit climbs
蓝筹股推动 马股升0.49%
KLCI rises 0.49% as select blue chips lift
国油相关股与国能上涨 支撑马股涨0.43%
KLCI pares gains, stays up 0.4

EA科技赔海事重工9128万
EA科技获2.39亿合约
EA科技获2.4亿租船订单
公私交流平台　經濟理事會有利國家長遠發展
制造业强稳力挺‧末季经济可成长4.8%
制造业强稳力挺‧末季经济可成长4.8%
EA科技
新合约贡献30%盈利
上任不到1年 EA Technique COO辞职
握9亿合约‧EA科技盈利稳定
EA科技赢9450万合约
EA技术获2合约值9450万
EA科技获9450万合约
EA科技迈向60仙/敏源
被海事重工索赔
EA科技遭追讨1.26亿
海事重工 仲裁案或需拨备
EA科技寻求仲裁
追海事重工9006万
EA科技挑战62仙/敏源
海事重工将反诉EA科技
马重船舶工程 遭EA技术索赔9000万
EA科技向海事重工追讨8995万
Amway 1Q earnings within expectations
蓝筹股推动 马股升0.49%
KLCI rises 0.49% as select blue chips lift
销量增·成本降·安利首季净利增33%
马股回吐涨幅 仍企于1600以上
KLCI pares gains, stays above 1,600 level as select blue chips lift
马股扩大跌幅
KLCI sheds 0.31%, extends downward consolidation
Consumer, property and auto sectors to benefit from rate cut
区域股市反弹 马股扬0.52%
KLCI rises 0.52% as regional markets stagger up
亚通数码网络走高 提振马股止跌回扬
KLCI reverses loss as Axiata, DiGi lift
成份股带动·马股上涨
怡克伟士价量齐升·马股收涨5.06点
马银行提振 马股止跌走高
KLCI reverses loss, up 0.18% as Maybank lifts
KLCI slips as HL Bank, Tenaga weigh
区域股市涨跌互见 马股小涨0.16%
KLCI up 0.16% as regional markets stay mixed
GDB控股进军打桩业务
GDB拟购打桩公司70％
成本规模虽大砍 东铁若重启 建筑股

Chip stocks under pressure
US-China trade war intensifies
Malaysia’s tech stocks hit multi-month lows amid escalating trade war
风险或机遇？
Risk or opportunity?
Malaysian tech stocks down after US blacklists Huawei
华为事件冲击‧大马科技股哀鸿遍野
华为禁令拖低那指 马科技股全线下跌
Quick take: Tech stocks tumble on Bursa Malaysia
A new large project of Globetronics likely to sustain its profit growth
Globetronics to see earnings improvement moving forward, says Affin Hwang
东益电子
次季盈利料反弹
The Week Ahead: Bank Negara’s OPR decision, China-US talks in focus
Uncertainty for corporate earnings as MFRS16 takes effect
Globetronics sensor business seen to pick up in April, mid-year
马股跟随区域股市跌0.14%
KLCI tracks muted region, dips 0.14%
F&N to produce original recipes in small pack sizes
KLCI drifts lower in line with regional markets
Globetronics to see higher sensor volumes on new product, says Affin Hwang
SC adds 14 companies to syariah list, removes 10
SC adds 14 companies to Shariah list, removes 10
Lower profit margin, revenue gro

Eye On Stock, Opensys
Ajiya, Hiap Huat, Opensys, Sanbumi, Sinotop
Hexza, K-One, OpenSys, PBA, Sanbumi, VisDynamics
Green Packet, Amverton, OpenSys, K-One, Merge, Reach, Minho, Jaycorp
Stock With Momentum: Opensys
傲奔系统再装500台存取款机
AWC, Opensys, Amverton, Deleum, Keck Seng, Lii Hen, Minho, Ni Hsin, Notion VTec, Pinehill
OpenSys推出现金循环机 带动股价上涨3.23%
OpenSys active, up 3.23% on plan to roll out Cash Recycling ATMs
Trading ideas: Pestech, Deleum, Ann Joo, Tan Chong, OpenSys
KLCI seen extending consolidation, firm crude oil to offer respite
KLCI seen extending consolidation, firm crude oil price to offer respite
Sapura Energy, ECM Libra, Hai-O, Velesto, Maxwell, Deleum, Pestech, OpenSys, Sumatec, KUB, V.S. Industry, Ann Joo, Maybulk, Tan Chong Motor and Warisan TC Holdings
OpenSys to roll out RM36m worth of OKI Cash Recycling ATMs in 3Q
Stock With Momentum: OpenSys (M) Bhd
Dancomech, EG, Mikro, Ancom, Far East, JF, Kim Teck Cheong, KUB, Minho, Lion Forest, OpenSys, Perak Transit
Support Line
冷眼心

...

The Brunei factor in Scomi
区域股市小涨 马股扬0.19%
KLCI up 0.19% in line with modest regional gains
Mild rebound in Tenaga lifts KLCI, RHB Bank climbs
马股跌至3年新低
KLCI drops 0.69% to lowest level in 3 years
区域股市稍事歇息 马股跌0.35%
KLCI dips 0.35% as regional markets pause
全球成长忧虑 拖累马股挫0.35%
KLCI dips 0.35% on global growth worries, FGV in focus ahead of White Paper
国能攀升助马股收窄跌幅
KLCI pares loss as Tenaga climbs
大马PMI跌至3个月新低 拖累马股由升转跌
KLCI reverses gains as Malaysia PMI falls to 3-month low
下跌股:Enra集团95仙支撑
ENRA 2820万英国发展房产
ENRA向Gemula索讨1038万
Enra integrates ICE with gas odorant ops
国能等蓝筹股领涨 马股止跌走高
KLCI reverses loss as Tenaga, select blue chips lift
ARK Resources Holdings, Cypark Resources, Genting and Tambun Indah Land
Mohamad Zekri Ibrahim is ARK Resources' new chairman
Trading seen muted ahead of extended weekend
APFT, Amanah Harta PNB, Bursa Malaysia, Chin Hin Group, Ark Resources, Berjaya Corp, Genting, Merge Energy, Uzma and Alam Maritim Resources
Ark Resources to develop 90-acre land in Balik Pulau
A

KLCI could stage technical rebound, move above 1,680
OSK Holdings, Genting, Gas Malaysia, SCH, MHB, Eversendai, N2N and Goodway
审计费谈不拢 顺行综合审计师呈辞
Auditors of Goodway Integrated Industries resign over audit fees
Immediate hurdle for Goodway at 28.5 sen, says AllianceDBS Research
Stock With Momentum: Goodway Integrated Industries
BCM Alliance, Dancomech, Dolphin, Formosa Prosonic, Goodway, Ni Hsin, Sinotop
顺行综合异军突起
区域股市回落 马股升幅收窄
KLCI pares gains as regional markets retreat
国油股走强 提振马股小涨
KLCI holds on to marginal gains as Petronas-linked stocks lift
順行綜合以750萬 售房產和機械
順行綜合以750萬售房產和機械
顺行综合工业680万卖厂
前景樂觀吸引購興 順行綜合衝一年新高
增长前景引购兴 顺行综合一度飙38%
Goodway up as growth outlook stirs buying interest
马股早盘休市微挫
KLCI stays in the red on negative market breadth
GLBhd, G Neptune, Jaycorp, Lafarge, Lay Hong, Leweko, Leader Steel
Building sustainable infrastructure
利维高湄公河天桥项目告吹
Leweko Resources drops collaboration with NUCETECH to build pedestrian bridges in Vietnam
利维高获3445万合约
更换旧桥工程
Poor April start at Wall Street

联合创办人之一
黎德坚任荣腾科技总执行长
联合创办人之一 黎德坚任荣腾科技总执行长
万通赢TIG亚太行销权
Hengyuan, CIMB, Hibiscus Petroleum, Barakah, Kawan Food, JAKS, Star Media, Scomi, Scomi Energy, MyEG, Aturmaju Resources and mTouche Technology
Trading ideas: MCE Holdings, CNI, mTouche, Plastrade
LPI Capital, Top Glove, Red Ideas, MCE, MAHB, Gamuda, MMC Corp, CNI Holdings and mTouche Technology
10月龙虎榜．848.85亿．国能市值创新高
MNC无线万通科技携手．为SPNB开发数码平台
加股潜在盈利．万通科技趋扬
万通科技偕A Conceptech
拓展O2O国际教育中心
马股升破1760点
KLCI ticks higher to breach 1,760 level
Lotte Chemical, Petronas Chemicals, Tenaga Nasional, Tropicana Corp, Uchi Technologies, Johan Holdings, Sime Darby, Bina Darulaman, Ranhill, Carimin Petroleum, Jaks Resources, and mTouche Technology
MNC无线偕万通科技
发展SPNB数字平台
MNC无线偕国家房屋
提供购屋者财务支援
小股交投热络．综指缓升
Tech stocks rise as FBM KLCI stages technical rebound
马股交投淡静‧综指全天微起0.15%
亚股下跌 马股走势低迷
KLCI stays lacklustre as Asian shares dip
OCK, Ucrest, Orion, Bahvest, Tri-Mode, IWCity, Hiap Teck, Marine & General, Can-One, SeaLink, Tambun Indah, BFood, PCCS, Master

KLCI seen hovering above 1,770-level on improved domestic stability
Gamuda, KPS, Bumi Armada, SLP Resources, Frontken, Axiata, Anzo, Lay Hong, AHP, Parlo, Nylex, Ancom, YTL Power, UEM Sunrise and T7 Global
Thirty Keystone to settle Parlo’s profit shortfall via release of 3.15% stake
KLCI to trade range bound within tight band, global trade war worries to persist
Maybank, Petaling Tin, KPS, UMW, Computer Forms, Parlo, Selangor Dredging, Pasdec, Tropicana and MyEG
Parlo says unit's PAT fell short of profit guarantee due to higher expenses
Parlo 16万买旅游公司
【独家】或须承担债务与问题 认清倒置收购危与机
Insider Moves: APFT Bhd, D'nonce Technology Bhd, Ipmuda Bhd, Lee Swee Kiat Group Bhd, Parlo Bhd, Privasia Technology Bhd
Stock With Momentum: Parlo Bhd
Parlo, Jadi, T7, RHB, Xidelang, Sern Kou
脱离GN3‧坝罗复牌状态勇
倒置收购取代赛柏塔Parlo盘中飙5.2倍
综指以1821.6点做收．跌7.03点
马股回吐涨幅
PARLO公司復牌大熱 股價一度飆2.1倍
投资者情绪转淡 马股小跌0.26%
KLCI down 0.26% as sentiment turns bearish
Small cap O&G counters see profit-taking, PetGas weighs
脱离GN3公司行列 刺激Parlo一度大涨21

成功媒体第三季净亏扩大
RGT脱离PN17
并购私有化好时机
成功媒体准迟交重组计划
陈志远有大动作．成功集团成功置地引购兴
企业重组利好催化 成功置地一度猛涨36%
成功机构拟卖京都四季酒店
放眼16.7亿收益
成功机构拟卖京都四季酒店 放眼16.7亿收益
不满估值偏低大重组 陈志远或私有化7-11
陈志远宣布连串企业计划 7-11下市 U Mobile上市
失望成功集团资产被低估‧陈志远或私有化7-11
“失望资产严重低估”．陈志远明年大动作
成功媒体次季亏257万
成功媒体 首半财年营业额跌25%
广告收入下降．成功媒体次季亏损恶化
成功媒体304万售成功集团0.23%
成功媒体首季亏275万
营收减成功媒体首季净亏275万
成功媒体重组计划
最迟12月20呈交
股东比例不达标陷PN17 成功媒体获准延交重组计划
Are printable buildings the way forward?
TNB CEO Amir Hamzah exits UEM Edgenta’s board
Ekuinas in talks to acquire subsidiaries of GLCs
Ekuinas eyes investment in GLC subsidiaries
Icon Offshore wins RM12.7m contract
Icon Offshore获Carigali Hess总值1270万合约
Icon岸外获1270万合约
Icon Offshore lands RM12.7m job from Carigali Hess
Icon gets RM12.7m vessel contract from Carigali Hess
Icon secures RM12.7m contract for vessel charter
陈建得加入善联控股董事部
Tan Kean Tet joins Sanbumi's board
标致创办人兼主席
陈建得任善联执董
标致集团创办人 任善联控股执行董事
画家起诉索偿100万
MYNEWS 否认侵权
MyNews says bag artwork was in public domain
Two artists sue Mynews, subsidiary for copyright breach
Two ar

## 4. Financial Reports
### Crawl financial reports -- PDF

https://www.malaysiastock.biz/Annual-Report.aspx

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
import os

def get_PDFurl():
    pdf_links = []
    links = soup.find_all(href=re.compile(".pdf"))
    for each in links:
        pdf_links.append(each.attrs['href'])
    return pdf_links


def get_pageurl():
    page_links = []
    links = soup.find_all(href=re.compile("javascript:__doPostBack"))
    for each in links:
        page_links.append(each.attrs['href'])
    return page_links


def download_pdf(pdf_links, current_page):
    print("Downloading Page " + current_page + " : ")
    j = 1
    # download automatically pdf files from website
    for i in pdf_links:
        file_name = i.split('&name=')[-1]
        r = requests.get(i)
        # create new folder (change it to your directory)
        if not os.path.exists('PDF/Page ' + current_page):
            os.mkdir('PDF/Page ' + current_page)
        with open('PDF/Page ' + current_page + '/' + file_name, 'wb') as pdf:
            pdf.write(r.content)
            print(str(j) + '--' + file_name + '...Successful!')
            j += 1
        pdf.close()
    print('Done With Page ' + current_page + '!\n\n')
    return


url = 'https://www.malaysiastock.biz/Annual-Report.aspx'

# Set browser environment
chrome_options = webdriver.ChromeOptions()
# Using headless mode to avroid connection error
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
browser = webdriver.Chrome(options=chrome_options)
browser.set_page_load_timeout(30)
browser.implicitly_wait(30)

# Download first page
browser.get(url)
time.sleep(3)
r = browser.page_source
soup = BeautifulSoup(r, 'html.parser')
pdf_links = get_PDFurl()
page_links = get_pageurl()
current_page = soup.find(class_='pgr').find('span').text
download_pdf(pdf_links, current_page)
time.sleep(2)

# Download all other pages
for i in page_links:
    browser.execute_script(i)  # turning page
    time.sleep(3)
    r = browser.page_source
    soup = BeautifulSoup(r, 'html.parser')
    pdf_links = get_PDFurl()
    current_page = soup.find(class_='pgr').find('span').text
    download_pdf(pdf_links, current_page)
    time.sleep(2)

browser.quit()
print("\n=====================ALL Done!!!=======================\n")

1--ILB-Corporate Governance Report 2018.pdf...Successful!
2--ILB Annual Report 2018.pdf...Successful!
3--Sustainability Report KPJ HEALTHCARE BERHAD 2018.pdf...Successful!
4--CG Report Final 2018.pdf...Successful!
5--Proxy Form KPJ HEALTHCARE BERHAD 2018.pdf...Successful!
6--Financial Report KPJ HEALTHCARE BERHAD 2018.pdf...Successful!
7--Integrated Report KPJ HEALTHCARE BERHAD 2018.pdf...Successful!
8--Ranhill Holdings Bhd_CG Report FY2018.pdf...Successful!
9--Ranhill Holdings Berhad AR18 - Part C.pdf...Successful!
10--Ranhill Holdings Berhad AR18 - Part B.pdf...Successful!
11--Ranhill Holdings Berhad AR18 - Part A.pdf...Successful!
12--CG Report (Bursa).pdf...Successful!
13--ATURMAJU PART 3.pdf...Successful!
14--ATURMAJU PART 2.pdf...Successful!
15--ATURMAJU PART 1.pdf...Successful!
16--Maybank CG Report_FY2018.pdf...Successful!
17--Maybank AR 2018_Financial.pdf...Successful!
18--Maybank AR 2018_Corporate-Part 2.pdf...Successful!
19--Maybank AR 2018_Corporate-Part 1.pdf...Successful!

17--PLB-AR2018.pdf...Successful!
18--Sasbadi CG Report 2018.pdf...Successful!
19--ANNUAL REPORT 2018-PART 2.pdf...Successful!
20--ANNUAL REPORT 2018-PART 1.pdf...Successful!
21--CG Report.pdf...Successful!
22--Annual Report 2018.pdf...Successful!
23--Melati Ehsan - CG Report.pdf...Successful!
24--Melati Ehsan- Annual Report.pdf...Successful!
25--BSL CORPORATION BERHAD-CG REPORT.pdf...Successful!
26--BSL CORPORATION BERHAD-ANNUAL REPORT 2018.pdf...Successful!
27--CG Report.pdf...Successful!
28--Annual Report 2018.pdf...Successful!
29--SWS Capital - AR 2018 (Part 2).pdf...Successful!
30--SWS Capital - AR 2018 (Part 1).pdf...Successful!
31--SWSCap - CG Report 2018.pdf...Successful!
32--CG Report.pdf...Successful!
33--SCH- Annual Report 2018.pdf...Successful!
34--Corporate Governance Report 2018.pdf...Successful!
35--Appendix A- Constitution.pdf...Successful!
36--Annual Report 2018.pdf...Successful!
37--Appendix A to Notice of 57th AGM (FNHB Proposed New Constitution).pdf...Successful!
38-

## 5. Forum Data
### Crawl forum information & comments

http://www.investalks.com/forum/forum.php?mod=forumdisplay&fid=7&typeid=17&filter=typeid&typeid=17&page=1

In [15]:
import requests
from lxml import etree
from lxml import html
import re
import time

header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36','Referer':'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-2305666475781689&output=html&h=90&slotname=9390552809&adk=3116781417&adf=1662620477&w=970&lmt=1552753083&guci=2.2.0.0.2.2.0.0&format=970x90&url=http%3A%2F%2Fwww.investalks.com%2Fforum%2Fforum.php%3Fmod%3Dforumdisplay%26fid%3D7%26filter%3Dtypeid%26typeid%3D17&flash=0&wgl=1&dt=1552753083912&bpp=42&bdt=108&fdt=46&idt=22&shv=r20190313&cbv=r20190131&saldr=aa&abxe=1&correlator=6683659723222&frm=20&pv=2&ga_vid=691979923.1552569252&ga_sid=1552752744&ga_hid=1232295072&ga_fc=1&iag=0&icsg=12206&dssz=10&mdo=0&mso=0&u_tz=480&u_his=3&u_java=0&u_h=864&u_w=1536&u_ah=824&u_aw=1536&u_cd=24&u_nplug=3&u_nmime=4&adx=34&ady=100&biw=1026&bih=350&scr_x=0&scr_y=0&eid=21060853&oid=3&rx=0&eae=0&fc=656&brdim=426%2C33%2C426%2C33%2C1536%2C0%2C1057%2C735%2C1042%2C350&vis=1&rsz=%7C%7CeE%7C&abl=CS&ppjl=f&pfx=0&fu=16&bc=7&ifi=1&uci=1.afatrway4czq&fsb=1&xpc=kkAgITwzNl&p=http%3A//www.investalks.com&dtd=90'}

num_list = []
hot_list = []
title_list = []
time_list = []
link_list = []

def get_page():
    url = 'http://www.investalks.com/forum/forum.php?mod=forumdisplay&fid=7&typeid=17&filter=typeid&typeid=17&page=1'
    r = requests.get(url, headers = header)
    time.sleep(2)
    html = etree.HTML(r.text)
    page = html.xpath('//div[@class="pg"]/a[@class="last"]/text()')[0].split(' ')[-1]
    return int(page)

def get_info(pages):
    for i in range(1,pages+1):
        html = 'http://www.investalks.com/forum/forum.php?mod=forumdisplay&fid=7&typeid=17&filter=typeid&typeid=17&page=' + str(i)
        html_i = requests.get(html, headers = header)
        time.sleep(2)
        selector =etree.HTML(html_i.text)
        content=selector.xpath('//table[@id="threadlisttableid"]/tbody')[2:]
        for tr in content:
            num_list.append(tr.xpath('.//td//a[@class="xi2"]//text()')[0])
            hot_list.append(tr.xpath('.//td[@class="num"]//em//text()')[0])
            title_list.append(tr.xpath('.//th//a[@class="s xst"]//text()')[0])
            time_list.append(tr.xpath('.//td//em//text()')[0])
            link_list.append('http://www.investalks.com/forum/'+tr.xpath('.//th//a[@class="s xst"]/@href')[0])
    return

total_page = get_page()
get_info(total_page)

import pandas as pd
dataframe = pd.DataFrame({'Titles':title_list,'Hot':hot_list,'Replies':num_list,'Time':time_list,'Link':link_list})
# dataframe.to_csv("Company Comments.csv",index=False,sep=',')
print(dataframe)


                                              Titles     Hot Replies  \
0                                 1961 IOICORP IOI集團  116941    1691   
1                            0020 NETX (前名 ARIANTEC)   49795     226   
2                                        4863 TM 马电讯  112946    1165   
3                                  8583 Mahsing 馬星集團  315421    2448   
4                              7084 QL Resource 全利资源  473814    2615   
5                                    5185 Affin 艾芬控股   79368     768   
6                                    5183 PCHEM 国油化工   99170     706   
7                             5210 Bumi Armada 布米阿马达  150056    1143   
8                                  5099 AirAsia 亞洲航空  587461    6019   
9                                   9261 GADANG 嘉登控股  256316    3016   
10                               5436 PERSTIMA 馬鍍錫工業   46750     421   
11                               5246 Westports 大马西港   56444     642   
12                                     3174 L&G 置地通用  291668    